<a href="https://colab.research.google.com/github/PivePipioipia/RAG/blob/main/RAG_Application_using_Langchain_OpenAI_API_and_FAISS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What is the RAG system?

##Defination:
This is called retrieval augmented generation (RAG), as you would retrieve the relevant data and used it as augmented context for the LLM. Instead of relying on knowlegde derived from the training data, a RAG workflow pulls relevant information and connects static LLMS with real-time data retrieval.
## Architecture (README)



# Why we create a RAG System?
Retrieval systems (RAG) give LLM systems access to factual, access-controlled, timely information


1. RAG REDUCES HALLUCINATION

Example: In the financial services industry, providing accurate information on investment options is crucial because it directly impacts customers' purchasing decisions and financial well-being. RAG can help ensure that the information generated about stocks, bonds, or mutual funds

2. COST-EFFECTIVE ALTERNATIVE

Example: Banks often need to assess the creditworthiness of potential borrowers. Fine-tuning pre-trained language models to analyse credit histories can be resource-intensive. RAG architecture offers a cost-effective alternative by retrieving relevant financial data and credit history information from existing databases, combining this with pre-trained language models

3. CREDIBLE AND ACCURATE RESPONSES

Example: In customer support, providing accurate and helpful responses is essential for maintaining customer trust, as it demonstrates the company's commitment to providing reliable information and support. The RAG technique is able to do this very effectively by retrieving data from catalogues, policies, and past customer interactions to generate context-aware insights, ensuring that customers receive reliable information on product features, returns, and other inquiries.

4. DOMAIN-SPECIFIC INFORMATION

Example: In the legal industry, clients often require advice specific to their case or jurisdiction because different legal systems have unique rules and regulations, and understanding these nuances is crucial for effective legal representation. RAG can access domain-specific knowledge bases, such as local statutes and case law, to provide tailored information relevant to clients' legal needs.




# RAG Practical Usecase


1.   Document Question Answering Systems
-> Hệ thống hỏi đáp trên hợp đồng, báo cáo, tài liệu kỹ thuật...
2.   Conversational Agents
-> Tác nhân hội thoại (chatbot thông minh)
(Ví dụ: chatbot hỗ trợ khách hàng có thể truy xuất dữ liệu thật)
3. Real-time Event Commentary
-> Bình luận sự kiện theo thời gian thực
(Ví dụ: bóng đá, chứng khoán, tin tức nóng...)
4. Content Generation
-> Sinh nội dung có kiểm chứng
(Ví dụ: viết báo, tóm tắt tài liệu, tạo báo cáo)
5. Personalised Recommendation
-> Gợi ý cá nhân hóa
(Ví dụ: gợi ý sản phẩm, khóa học, phim ảnh phù hợp với từng người)
6. Virtual Assistants
-> Trợ lý ảo
(Ví dụ: trợ lý công việc, trợ lý học tập, trợ lý mua sắm...)


# Installing the necessary libraries

In [2]:
!pip install langchain openai tiktoken rapidocr-onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 66.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.0 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# Setting Enviroment Variable

In [4]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY



1.   Data Ingestion
2.   Data Retrieval
3.   Data Generation



# Data Ingestion

https://en.wikipedia.org/wiki/State_of_the_Union#:~:text=Though%20the%20language%20of%20the,as%20late%20as%20March%207

In [5]:
!pip install -q langchain langchain-community faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [6]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS

In [7]:
from google.colab import files
uploaded = files.upload()

Saving einstein.txt to einstein.txt


In [8]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("einstein.txt", encoding = "utf8")
document = loader.load()

print(document[0].page_content[:500])

Albert Einstein, one of the most influential scientists in human history, was born on March 14, 1879, in the city of Ulm, in the Kingdom of Württemberg in the German Empire. His early life was not extraordinary; in fact, teachers once believed he was slow because he spoke relatively late compared to other children. However, from a young age, Einstein displayed a deep curiosity about nature and an unusual ability to understand complex concepts in mathematics and physics.

Einstein’s family moved 


# Chunking of the Data
Text splitter which is available in Langchain
https://python.langchain.com/docs/how_to/#text-splitters

CharacterTextSplitter v/s RecursiveCharacterTextSplitter
https://chunkviz.up.railway.app/

In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter



In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

In [11]:
text_chunks = text_splitter.split_documents(document[:1])

In [12]:
print(text_chunks[3].page_content)

Over the following years, Einstein held academic positions in Zurich, Prague, and Berlin. In 1921, he received the Nobel Prize in Physics for his explanation of the photoelectric effect, a discovery that laid the foundation for quantum theory. While he was celebrated worldwide as a genius, Einstein remained deeply humble and committed to using science for the betterment of humanity. During the rise of the Nazi regime in Germany, Einstein, who was Jewish, emigrated to the United States in 1933. He accepted a position at Princeton University, where he remained for the rest of his career.


In [13]:
import textwrap

wrapped_text = textwrap.fill(text_chunks[3].page_content, width=100)
print(wrapped_text)


Over the following years, Einstein held academic positions in Zurich, Prague, and Berlin. In 1921,
he received the Nobel Prize in Physics for his explanation of the photoelectric effect, a discovery
that laid the foundation for quantum theory. While he was celebrated worldwide as a genius, Einstein
remained deeply humble and committed to using science for the betterment of humanity. During the
rise of the Nazi regime in Germany, Einstein, who was Jewish, emigrated to the United States in
1933. He accepted a position at Princeton University, where he remained for the rest of his career.


In [14]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [15]:
!pip install langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 3.8 MB/s eta 0:00:00


In [16]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_API_KEY
)


In [17]:
!pip install faiss-cpu

In [18]:
!pip install sentence-transformers transformers accelerate


In [19]:
from huggingface_hub import login
from google.colab import userdata

HF_TOKEN = userdata.get('HF_TOKEN')

login(token=HF_TOKEN)

In [20]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS



In [22]:
embeddings = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [23]:
vectorstore = FAISS.from_documents(text_chunks, embeddings)
retriever = vectorstore.as_retriever()

# Prompt Templete

In [34]:
from langchain.prompts import ChatPromptTemplate

In [45]:
template = """You are an assistant for question-answering tasks.
Use the following pieces of retriever context to answer the question.
If you don't know the answer, just say you don't know.
Keep the answer concise.
Do not repeat the context or the instructions. Just give the answer.


Question: {question}
Context: {context}
Answer:
"""



In [46]:
prompt = ChatPromptTemplate.from_template(template)

In [47]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_community.llms import HuggingFacePipeline

In [38]:

model_id = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


In [48]:
llm_model = HuggingFacePipeline(pipeline = pipe)

In [49]:
output_parser=StrOutputParser()

In [50]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [51]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [52]:
print(rag_chain.invoke("When and where was Albert Einstein born?"))


Human: You are an assistant for question-answering tasks.
Use the following pieces of retriever context to answer the question.
If you don't know the answer, just say you don't know.
Keep the answer concise.
Do not repeat the context or the instructions. Just give the answer.


Question: When and where was Albert Einstein born?
Context: [Document(id='ef84fec3-e384-4272-b537-8fbf193af311', metadata={'source': 'einstein.txt'}, page_content='Albert Einstein, one of the most influential scientists in human history, was born on March 14, 1879, in the city of Ulm, in the Kingdom of Württemberg in the German Empire. His early life was not extraordinary; in fact, teachers once believed he was slow because he spoke relatively late compared to other children. However, from a young age, Einstein displayed a deep curiosity about nature and an unusual ability to understand complex concepts in mathematics and physics.'), Document(id='8b4d4a6e-5873-4d24-b6ff-278367b631fb', metadata={'source': 'einste

In [53]:
print(rag_chain.invoke("Why did Einstein leave school at the age of 15?"))


Human: You are an assistant for question-answering tasks.
Use the following pieces of retriever context to answer the question.
If you don't know the answer, just say you don't know.
Keep the answer concise.
Do not repeat the context or the instructions. Just give the answer.


Question: Why did Einstein leave school at the age of 15?
Context: [Document(id='00871baa-1e57-4acf-ab36-0fc4c4a343b8', metadata={'source': 'einstein.txt'}, page_content='Einstein’s family moved to Munich, where he attended elementary school and later the Luitpold Gymnasium. Although he excelled in mathematics, he disliked the rigid discipline of the German school system. At the age of 15, Einstein left school and joined his parents in Italy. Soon afterward, he enrolled at the Swiss Federal Polytechnic School in Zurich, where he trained to become a teacher in physics and mathematics. It was during these years that Einstein began to develop his independent and creative approach to scientific thinking.'), Document

In [54]:
print(rag_chain.invoke("What famous equation did Einstein introduce in 1905?"))


Human: You are an assistant for question-answering tasks.
Use the following pieces of retriever context to answer the question.
If you don't know the answer, just say you don't know.
Keep the answer concise.
Do not repeat the context or the instructions. Just give the answer.


Question: What famous equation did Einstein introduce in 1905?
Context: [Document(id='22528ea3-4b97-4417-b44b-ff414e25dd10', metadata={'source': 'einstein.txt'}, page_content='After graduating in 1900, Einstein struggled to find a teaching job. He eventually accepted a position at the Swiss Patent Office in Bern. This seemingly unremarkable job gave him plenty of free time to work on scientific ideas. In 1905, often referred to as his "Annus Mirabilis" or "Miracle Year," Einstein published four groundbreaking papers in the journal Annalen der Physik. These papers covered the photoelectric effect, Brownian motion, special relativity, and the equivalence of mass and energy, introducing the famous equation E=mc². T